# <b>Price Prediction using XGBOOST

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import yfinance as yf

In [3]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S%26P_500_component_stocks')
table_symbol = data[0]
table_industry = data[0]['GICS Sector']

#these 2 lists are in order with one another
symbols = list(table_symbol.Symbol.values)
industries = list(table_industry.values) ##Industry Sector

remove = []

symbols_to_remove = ['BF.B', 'BRK.B', 'KVUE', 'VLTO','SPY']

for i, symbol in enumerate(symbols):
    if symbol in symbols_to_remove:
        remove.append(i)

for i in remove:
    symbols.remove(symbols[i])
    industries.remove(industries[i])

print(len(symbols))

symbol_industry = dict(zip(symbols,industries))
print(len(symbol_industry))

499
499


In [4]:
tickers = yf.Tickers('AAPL')

APPLdf = tickers.tickers['AAPL'].history(period="5d", start="2022-01-01", end="2022-12-31")
APPLdf.head(10)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00-05:00,175.821100,180.814055,175.702461,179.953873,104487900,0.0,0.0
2022-01-04 00:00:00-05:00,180.566860,180.873356,177.096503,177.669952,99310400,0.0,0.0
2022-01-05 00:00:00-05:00,177.581022,178.134693,172.667164,172.944000,94537600,0.0,0.0
2022-01-06 00:00:00-05:00,170.749066,173.319700,169.701043,170.056976,96904000,0.0,0.0
2022-01-07 00:00:00-05:00,170.936919,172.172799,169.097931,170.225052,86709100,0.0,0.0
2022-01-10 00:00:00-05:00,167.169974,170.551338,166.270251,170.244843,106765600,0.0,0.0
2022-01-11 00:00:00-05:00,170.373372,173.201049,168.890317,173.102188,76138300,0.0,0.0
2022-01-12 00:00:00-05:00,174.130420,175.178443,172.845118,173.547089,74805200,0.0,0.0
2022-01-13 00:00:00-05:00,173.794284,174.624791,169.849352,170.244843,84505800,0.0,0.0


In [5]:
drop_cols = ['Open', 'Low', 'High','Volume', 'Dividends', 'Stock Splits']
APPLdf = APPLdf.drop(columns=drop_cols, axis = 1)
APPLdf.index = APPLdf.index.astype(str).str.split(' ').str[0]
APPLdf

,Close
Date,
2022-01-03,179.953873
2022-01-04,177.669952
2022-01-05,172.944000
2022-01-06,170.056976
2022-01-07,170.225052
...,...
2022-12-23,131.127060
2022-12-27,129.307236
2022-12-28,125.339417


In [28]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=1, subplot_titles=('Price of APPL for 2022',))

# Add trace for Close price
fig.add_trace(go.Scatter(x=APPLdf.index, y=APPLdf['Close'], name='Close Price'), row=1, col=1)

# Get unique months in the DataFrame
unique_months = pd.to_datetime(APPLdf.index).to_period('M').unique()

# Update x-axis layout to show ticks for each month
fig.update_xaxes(
    tickmode='array',
    tickvals=unique_months.to_timestamp(),  # Convert to timestamp for plotting
    ticktext=unique_months.strftime('%b'),  # Displaying abbreviated month names
)
# Show figure
fig.show()

In [29]:
holidays = ["2022-01-17","2022-02-21","2022-04-15", "2022-05-30", "2022-06-20", "2022-07-4", "2022-09-5", "2022-11-24","2022-12-26"]
len(holidays)

9

In [46]:
holiday_df = pd.DataFrame(index=holidays)
holiday_df['Date'] = holiday_df.index

# Add another column 'Close' with NaN values
holiday_df['Close'] = float('nan')

# Set the 'Date' column as the index
holiday_df.set_index('Date', inplace=True)

# Print the resulting DataFrame
holiday_df

,Close
Date,
2022-01-17,NaN
2022-02-21,NaN
2022-04-15,NaN
2022-05-30,NaN
2022-06-20,NaN
2022-07-4,NaN
2022-09-5,NaN
2022-11-24,NaN
2022-12-26,NaN


In [47]:
APPLdf.loc["2022-11-25"]

Close    147.286743
Name: 2022-11-25 00:00:00, dtype: float64

In [49]:
# Assuming APPLdf and holiday_df are DataFrames

# Concatenate the original DataFrame with the holiday DataFrame
frames = [APPLdf, holiday_df]
result_df = pd.concat(frames)

# Convert the index to a consistent data type (e.g., Timestamp)
result_df.index = pd.to_datetime(result_df.index)

# Sort the index to maintain order
result_df = result_df.sort_index()

# Print the resulting DataFrame
APPL1 = result_df
APPL1


,Close
Date,
2022-01-03,179.953873
2022-01-04,177.669952
2022-01-05,172.944000
2022-01-06,170.056976
2022-01-07,170.225052
...,...
2022-12-26,NaN
2022-12-27,129.307236
2022-12-28,125.339417


In [61]:
# Convert index to datetime if it's in string format
APPL1.index = pd.to_datetime(APPL1.index)

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame(index=[f'APPL Week {week}' for week in range(1, 53)],
                      columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'])

# Iterate through rows of the original DataFrame and fill the new DataFrame
for i, row in APPL1.iterrows():
    week = i.week
    day = i.dayofweek
    
    # If there are rows for the current week and day, set the mean close price
    if not pd.isna(row['Close']):
        new_df.at[f'APPL Week {week}', f'Day {day + 1}'] = row['Close']

# Display the new DataFrame
APPL = new_df
nan_rows = new_df[new_df.isna().any(axis=1)]
nan_rows

,Day 1,Day 2,Day 3,Day 4,Day 5
APPL Week 3,NaN,167.881851,164.352158,162.651596,160.575302
APPL Week 8,NaN,162.670731,158.463409,161.106598,163.195404
APPL Week 15,164.086365,165.977188,168.689697,163.630966,NaN
APPL Week 22,NaN,147.562607,147.433701,149.912262,144.132294
APPL Week 25,NaN,134.703903,134.18837,137.083298,140.444199
APPL Week 27,NaN,140.345062,141.69339,145.093948,145.77803
APPL Week 36,NaN,153.41655,154.836273,153.347061,156.236084
APPL Week 47,147.187302,149.345215,150.230286,NaN,147.286743
APPL Week 52,NaN,129.307236,125.339417,128.889572,129.207794


In [66]:
# Convert index to datetime if it's in string format
APPL1.index = pd.to_datetime(APPL1.index)

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame(index=[f'APPL Week {week}' for week in range(1, 53)],
                      columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'])

# Iterate through rows of the original DataFrame and fill the new DataFrame
for i, row in APPL1.iterrows():
    week = i.week
    day = i.dayofweek
    
    # If there are rows for the current week and day, set the mean close price
    if not pd.isna(row['Close']):
        new_df.at[f'APPL Week {week}', f'Day {day + 1}'] = row['Close']

# Drop rows with NaN values
new_df = new_df.dropna()

# Display the new DataFrame
APPL = new_df
APPL = APPL.drop(['APPL Week 1'], axis=0)
APPL


,Day 1,Day 2,Day 3,Day 4,Day 5
APPL Week 2,170.244843,173.102188,173.547089,170.244843,171.114899
APPL Week 4,159.794235,157.975021,157.886032,157.421341,168.405838
APPL Week 5,172.805557,172.637497,173.853607,170.946793,170.659729
APPL Week 6,169.937057,173.075241,174.510666,170.39241,166.947357
APPL Week 7,167.184982,171.055695,170.818115,167.184982,165.620804
APPL Week 9,163.462662,161.561966,164.888245,164.561554,161.532272
APPL Week 10,157.701126,155.859787,161.314484,156.92894,153.176987
APPL Week 11,149.108246,153.533356,157.988174,159.007874,162.334122
APPL Week 12,163.720093,167.125549,168.501633,172.322861,172.966339
APPL Week 13,173.837509,177.163757,175.985733,172.857452,172.560471


In [69]:
APPLPCT1 = APPL1.pct_change()
APPLPCT1

/var/folders/bq/q6bbfq_s68l_m1x6yxgvqnt80000gn/T/ipykernel_42007/1966763988.py:1: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.



,Close
Date,
2022-01-03,NaN
2022-01-04,-0.012692
2022-01-05,-0.026600
2022-01-06,-0.016693
2022-01-07,0.000988
...,...
2022-12-26,0.000000
2022-12-27,-0.013878
2022-12-28,-0.030685


In [74]:
# Convert index to datetime if it's in string format
APPLPCT1.index = pd.to_datetime(APPL1.index)

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame(index=[f'APPL PC Week {week}' for week in range(1, 53)],
                      columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'])

# Iterate through rows of the original DataFrame and fill the new DataFrame
for i, row in APPLPCT1.iterrows():
    week = i.week
    day = i.dayofweek
    
    # If there are rows for the current week and day, set the mean close price
    if not pd.isna(row['Close']):
        new_df.at[f'APPL PC Week {week}', f'Day {day + 1}'] = row['Close']

new_df.replace(0, pd.NA, inplace=True)

# Drop rows with NaN values
new_df = new_df.dropna()

# Display the new DataFrame
APPLPCT = new_df
APPLPCT

,Day 1,Day 2,Day 3,Day 4,Day 5
APPL PC Week 2,0.000116,0.016784,0.002570,-0.019028,0.005111
APPL PC Week 4,-0.004864,-0.011385,-0.000563,-0.002943,0.069778
APPL PC Week 5,0.026126,-0.000973,0.007044,-0.01672,-0.001679
APPL PC Week 6,-0.004235,0.018467,0.008294,-0.023599,-0.020218
APPL PC Week 7,0.001423,0.023152,-0.001389,-0.021269,-0.009356
APPL PC Week 9,0.001638,-0.011628,0.020588,-0.001981,-0.018408
APPL PC Week 10,-0.023718,-0.011676,0.034997,-0.027186,-0.023909
APPL PC Week 11,-0.026562,0.029677,0.029015,0.006454,0.020919
APPL PC Week 12,0.008538,0.020800,0.008234,0.022678,0.003734
APPL PC Week 13,0.005037,0.019134,-0.006649,-0.017776,-0.001718


In [78]:
all = pd.DataFrame(columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'])

# Iterate through each symbol
for symbol in symbols:
    # Fetch historical stock data for the symbol
    ticker = yf.Ticker(symbol)
    stock_data = ticker.history(period="1y")  # Fetch data for the last 1 year

    # Extract the close prices for each week
    for week in range(1, 53):
        start_date = f"2022-01-01"  # Change the start date as needed
        end_date = f"2022-12-31"    # Change the end date as needed
        week_data = stock_data[start_date:end_date]

        # If data for the current week is missing, create a row with NaN values
        if len(week_data) < 5:
            week_row = pd.DataFrame([[np.nan] * 5], columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'],
                                     index=[f'{symbol} Week {week}'])
        else:
            week_row = pd.DataFrame([week_data['Close'].tolist()[:5]], columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'],
                                     index=[f'{symbol} Week {week}'])

        # Concatenate the temporary DataFrame with the result DataFrame
        all = pd.concat([all, week_row])

# Display the resulting DataFrame
all

/var/folders/bq/q6bbfq_s68l_m1x6yxgvqnt80000gn/T/ipykernel_42007/2891734177.py:24: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

BF.B: No price data found, symbol may be delisted (period=1y)


,Day 1,Day 2,Day 3,Day 4,Day 5
MMM Week 1,117.534454,118.939507,118.788628,118.807487,119.750481
MMM Week 2,117.534454,118.939507,118.788628,118.807487,119.750481
MMM Week 3,117.534454,118.939507,118.788628,118.807487,119.750481
MMM Week 4,117.534454,118.939507,118.788628,118.807487,119.750481
MMM Week 5,117.534454,118.939507,118.788628,118.807487,119.750481
...,...,...,...,...,...
ZTS Week 48,146.791000,146.513519,152.746674,155.224060,155.997009
ZTS Week 49,146.791000,146.513519,152.746674,155.224060,155.997009
ZTS Week 50,146.791000,146.513519,152.746674,155.224060,155.997009
ZTS Week 51,146.791000,146.513519,152.746674,155.224060,155.997009


In [106]:
all.iloc[1::52]

,Day 1,Day 2,Day 3,Day 4,Day 5
MMM Week 2,117.534454,118.939507,118.788628,118.807487,119.750481
AOS Week 2,58.679440,59.396244,59.641720,59.386425,59.288231
ABT Week 2,103.323036,102.940681,105.470085,105.813217,105.970070
ABBV Week 2,152.278244,152.057190,154.921478,155.354004,157.305176
ACN Week 2,286.055237,283.476196,296.223694,298.093903,295.436157
...,...,...,...,...,...
YUM Week 2,124.114693,123.712273,126.283890,126.804100,127.285034
ZBRA Week 2,269.000000,266.820007,270.279999,268.820007,274.859985
ZBH Week 2,116.775383,118.095261,119.186897,122.412193,122.035088
ZION Week 2,47.835232,48.597088,49.349426,49.511326,49.758923


In [76]:
naners = all[all.isna().any(axis=1)].index.tolist()
print(naners[::52])
print(len(naners))
#12 stocks

['BF.B Week 1', 'KVUE Week 1', 'VLTO Week 1']
156


In [87]:
nan_ = all[all.isna().any(axis=1)]
nan_.iloc[::52]
removal = ['BF.B','KVUE', 'VLTO']

,Day 1,Day 2,Day 3,Day 4,Day 5
BF.B Week 1,NaN,NaN,NaN,NaN,NaN
KVUE Week 1,NaN,NaN,NaN,NaN,NaN
VLTO Week 1,NaN,NaN,NaN,NaN,NaN


In [91]:
temp = all.T.drop(columns=naners, axis = 1)
sp500_52weeks = temp.T

(25948, 5)

In [90]:
sp500_52weeks.shape

(25792, 5)

In [92]:
sp500_52weeks.iloc[::52]

,Day 1,Day 2,Day 3,Day 4,Day 5
MMM Week 1,117.534454,118.939507,118.788628,118.807487,119.750481
AOS Week 1,58.679440,59.396244,59.641720,59.386425,59.288231
ABT Week 1,103.323036,102.940681,105.470085,105.813217,105.970070
ABBV Week 1,152.278244,152.057190,154.921478,155.354004,157.305176
ACN Week 1,286.055237,283.476196,296.223694,298.093903,295.436157
...,...,...,...,...,...
YUM Week 1,124.114693,123.712273,126.283890,126.804100,127.285034
ZBRA Week 1,269.000000,266.820007,270.279999,268.820007,274.859985
ZBH Week 1,116.775383,118.095261,119.186897,122.412193,122.035088
ZION Week 1,47.835232,48.597088,49.349426,49.511326,49.758923


In [93]:
# Get the list of S&P 500 stocks
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500_tickers = sp500['Symbol'].tolist()

# Stocks to exclude
removal = ['BF.B', 'KVUE', 'VLTO', 'SP500']

# Fetch historical data for each stock
dfs = []
for ticker in sp500_tickers:
    if ticker not in removal:
        try:
            stock_data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
            stock_data = stock_data['Close'].reset_index()
            stock_data['Ticker'] = ticker  # Add a column for the stock ticker
            dfs.append(stock_data)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

# Concatenate the DataFrames
sp500_df = pd.concat(dfs, ignore_index=True)

# Display the resulting DataFrame
print(sp500_df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******


1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

/var/folders/bq/q6bbfq_s68l_m1x6yxgvqnt80000gn/T/ipykernel_42007/1705546249.py:21: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [101]:
sp500_df

,Date,Close,Ticker
0,2022-01-03,177.740005,MMM
1,2022-01-04,180.229996,MMM
2,2022-01-05,179.490005,MMM
3,2022-01-06,178.000000,MMM
4,2022-01-07,179.949997,MMM
...,...,...,...
124993,2022-12-23,145.759995,ZTS
124994,2022-12-27,145.300003,ZTS
124995,2022-12-28,143.830002,ZTS
124996,2022-12-29,148.149994,ZTS


In [104]:
sp500_df.set_index(['Date'])

,Close,Ticker
Date,,
2022-01-03,177.740005,MMM
2022-01-04,180.229996,MMM
2022-01-05,179.490005,MMM
2022-01-06,178.000000,MMM
2022-01-07,179.949997,MMM
...,...,...
2022-12-23,145.759995,ZTS
2022-12-27,145.300003,ZTS
2022-12-28,143.830002,ZTS


In [105]:
b = sp500_df.pct_change()
b

TypeError: cannot perform __truediv__ with this index type: DatetimeArray